# Logbook

In [4]:
import sys
import os
sys.path.append('..')
from src.data import nc_to_obs
filename = os.path.join('..', 'obs', 'les_Bruno', 'outdir_0005', 'profiles.nc')
test_obs = nc_to_obs(filename)

In [9]:
test_obs.values

<xarray.Dataset> Size: 3MB
Dimensions:  (z: 256, t: 241)
Coordinates:
  * z        (z) float32 1kB -598.8 -596.5 -594.1 ... -5.859 -3.516 -1.172
  * t        (t) float32 964B 0.0 3.6e+03 7.2e+03 ... 8.604e+05 8.64e+05
Data variables:
    b        (t, z) float32 247kB ...
    u        (t, z) float32 247kB ...
    v        (t, z) float32 247kB ...
    w        (t, z) float32 247kB ...
    ke       (t, z) float32 247kB ...
    pe       (t, z) float32 247kB ...
    eke      (t, z) float32 247kB ...
    wb       (t, z) float32 247kB ...
    wpbp     (t, z) float32 247kB ...
    wpup     (t, z) float32 247kB ...
    bpbp     (t, z) float32 247kB ...

In [ ]:
desc_ex = 